## PART 1 – Commodity Futures Time-Series Momentum Strategy

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import skew, kurtosis
import os

# Load the data
file_path = "/content/sample_data/Research Project Data.xlsx"

data_in = pd.read_excel(
    file_path,
    sheet_name='Futures Returns',
    usecols='A:Z'
)
data_oos = pd.read_excel(
    file_path,
    sheet_name='Futures Out of Sample',
    usecols='A:Z'
)

# Convert YearMonth to datetime
data_in['Date'] = pd.to_datetime(data_in['YearMonth'].astype(str), format='%Y%m')
data_in.set_index('Date', inplace=True)
data_oos['Date'] = pd.to_datetime(data_oos['YearMonth'].astype(str), format='%Y%m')
data_oos.set_index('Date', inplace=True)
# convert and set Date index
data_in['Date'] = pd.to_datetime(data_in['YearMonth'].astype(str), format='%Y%m')
data_in.set_index('Date', inplace=True)
data_oos['Date'] = pd.to_datetime(data_in['YearMonth'].astype(str), format='%Y%m')
data_oos.set_index('Date', inplace=True)

# now drop the original YearMonth column
data_in = data_in.drop(columns=['YearMonth'])
data_oos = data_oos.drop(columns=['YearMonth'])

### 1. Long-only Stats for In-Sample Period

In [ ]:
def summary_stats(df):
    stats = pd.DataFrame(index=df.columns)
    stats['Mean Ann Return'] = df.mean() * 12 * 100
    stats['Volatility'] = df.std() * np.sqrt(12) * 100
    stats['Sharpe'] = stats['Mean Ann Return'] / stats['Volatility']
    return round(stats, 2)

long_only_stats_in = summary_stats(data_in)

print("Stat Summary for In-Sample Period (Mean Returns and Volatility shown as %)\n")
print(long_only_stats_in)

Stat Summary for In-Sample Period (Mean Returns and Volatility shown as %)

               Mean Ann Return  Volatility  Sharpe
Corn                     14.41       29.92    0.48
Kansas Wheat             12.01       30.23    0.40
Soybeans                 13.76       28.63    0.48
Wheat                    12.71       31.44    0.40
Brent Crude              25.34       31.77    0.80
WTI Crude                24.40       34.52    0.71
Heating Oil              24.66       34.01    0.73
Gasoil                   25.18       33.17    0.76
Natural Gas              22.91       60.19    0.38
Gasoline                 25.78       40.06    0.64
Cocoa                    10.08       33.85    0.30
Cotton                   10.06       34.63    0.29
Coffee                   12.03       34.24    0.35
Sugar                    17.79       39.13    0.45
Feeder Cattle             7.78       15.03    0.52
Live Cattle               7.48       16.65    0.45
Lean Hogs                17.10       34.22    0.50
Gold  

### 2. Time-series Momentum Strategy - Individual Contracts

In [ ]:
returns = data_in

results_individual = []
lookbacks = [3, 12]

for lb in lookbacks:
    # compute lagged cumulative return over lookback (exclude current month)
    cum_ret = returns.shift(1).rolling(window=lb).apply(
        lambda x: np.prod(1 + x) - 1, raw=True
    )
    # momentum signal
    signals = np.sign(cum_ret)
    # raw strategy returns per contract
    strat_raw = signals * returns
    # leverage each contract to 40% annualized volatility
    vol_i = strat_raw.std() * np.sqrt(12)
    mult_i = 0.4 / vol_i
    strat_levered = strat_raw * mult_i

    for contract in strat_levered.columns:
        s = strat_levered[contract].dropna()
        ann_ret = s.mean() * 12 * 100
        ann_vol = s.std() * np.sqrt(12) * 100
        sharpe = ann_ret / ann_vol
        skew = s.skew()
        kurtosis = s.kurt() + 3  # convert to raw kurtosis
        results_individual.append({
            'Lookback': f'{lb}-month',
            'Contract': contract,
            'Annualized Return': ann_ret,
            'Annualized Volatility': ann_vol,
            'Sharpe Ratio': sharpe,
            'Skewness': skew,
            'Kurtosis': kurtosis
        })

df_output_individual = pd.DataFrame(results_individual).set_index(['Lookback', 'Contract']).sort_index()

print('Annualized Returns and Volatility shown as %')
df_output_individual

Annualized Returns and Volatility shown as %


Annualized Return  Annualized Volatility  \
Lookback Contract                                                  
12-month Aluminum                5.841980                   40.0   
         Brent Crude            14.333428                   40.0   
         Cocoa                   0.128957                   40.0   
         Coffee                 11.006563                   40.0   
         Copper                 15.830706                   40.0   
         Corn                    2.458110                   40.0   
         Cotton                  2.548580                   40.0   
         Feeder Cattle          10.194924                   40.0   
         Gasoil                 14.878617                   40.0   
         Gasoline               14.636847                   40.0   
         Gold                   31.683260                   40.0   
         Heating Oil            16.933033                   40.0   
         Kansas Wheat            4.710760                   40.0   
         Lead                    7.475788                   40.0   
         Lean Hogs              -2.471686                   40.0   
         Live Cattle            -0.360011                   40.0   
         Natural Gas             3.182239                   40.0   
         Nickel                 10.145420                   40.0   
         SPX                    22.849299                   40.0   
         Silver                 17.163508                   40.0   
         Soybeans                1.746144                   40.0   
         Sugar                   2.482583                   40.0   
         WTI Crude              10.235593                   40.0   
         Wheat                  -1.619931                   40.0   
         Zinc                   23.560380                   40.0   
3-month  Aluminum               25.342770                   40.0   
         Brent Crude            23.977239                   40.0   
         Cocoa                   2.068603                   40.0   
         Coffee                  1.274387                   40.0   
         Copper                 25.577205                   40.0   
         Corn                   14.704574                   40.0   
         Cotton                  6.035857                   40.0   
         Feeder Cattle          16.778576                   40.0   
         Gasoil                 31.282759                   40.0   
         Gasoline               15.091400                   40.0   
         Gold                   13.363502                   40.0   
         Heating Oil            28.110828                   40.0   
         Kansas Wheat            4.588290                   40.0   
         Lead                   15.093291                   40.0   
         Lean Hogs               6.984594                   40.0   
         Live Cattle             1.544164                   40.0   
         Natural Gas             2.873075                   40.0   
         Nickel                 26.408733                   40.0   
         SPX                    12.359210                   40.0   
         Silver                  3.907024                   40.0   
         Soybeans                7.903120                   40.0   
         Sugar                  12.145886                   40.0   
         WTI Crude              18.690085                   40.0   
         Wheat                  -0.192100                   40.0   
         Zinc                   31.752596                   40.0   

                        Sharpe Ratio  Skewness  Kurtosis  
Lookback Contract                                         
12-month Aluminum           0.146049 -0.017269  3.574492  
         Brent Crude        0.358336 -0.724233  4.278083  
         Cocoa              0.003224 -0.141292  4.074602  
         Coffee             0.275164  0.132270  3.713533  
         Copper             0.395768  0.585980  5.418252  
         Corn               0.061453  0.000679  2.958155  
         Cotton

### 3. Time-series Momentum Strategy - Equal Weighted Portfolio

In [ ]:
results_eq_wgt = []
returns = data_in
for lb in lookbacks:
    # compute lagged cum ret & signal
    cum_ret = returns.shift(1).rolling(window=lb).apply(
        lambda x: np.prod(1 + x) - 1, raw=True
    )
    signals = np.sign(cum_ret)
    strat_raw = signals * returns

    # equally-weighted portfolio raw returns
    port_raw = strat_raw.mean(axis=1)

    # leverage portfolio to 40% annualized volatility
    port_vol = port_raw.std() * np.sqrt(12)
    port_mult = 0.4 / port_vol
    port_levered = port_raw * port_mult

    # compute metrics
    ann_ret = port_levered.mean() * 12
    ann_vol = port_levered.std() * np.sqrt(12)
    sharpe = ann_ret / ann_vol
    skew = port_levered.skew()
    kurtosis = port_levered.kurt() + 3
    results_eq_wgt.append({
        'Lookback': f'{lb}-month',
        'Annualized Return': ann_ret * 100,
        'Annualized Volatility': ann_vol * 100,
        'Sharpe Ratio': sharpe,
        'Skewness': skew,
        'Kurtosis': kurtosis
    })

print("Annualized returns and volatility are shown in %")
df_output_eq_wgt = pd.DataFrame(results_eq_wgt).set_index('Lookback')
df_output_eq_wgt

Annualized returns and volatility are shown in %


,Annualized Return,Annualized Volatility,Sharpe Ratio,Skewness,Kurtosis
Lookback,,,,,
3-month,36.619423,40.0,0.915486,1.014265,8.665369
12-month,22.427663,40.0,0.560692,-0.591501,5.461240


### 4. Two-factor regression on portfolio returns

In [ ]:
data1 = pd.read_excel(
    file_path,
    sheet_name='Futures Returns',
    usecols='A:AB'
)


data1['Date'] = pd.to_datetime(data1['YearMonth'].astype(str), format='%Y%m')
data1.set_index('Date', inplace=True)
data1= data1.drop(columns=['YearMonth'])

In [ ]:
data1 = data1.rename(columns={data1.columns[-2]: 'Mkt_RF', data1.columns[-1]: 'CI_Ret'})

# Split futures returns and factors
fut_returns = data1.drop(columns=['Mkt_RF', 'CI_Ret','SPX'])
dep_factors = data1[['Mkt_RF', 'CI_Ret']]

def compute_port_levered(returns, lookback):
    cum_ret = returns.shift(1).rolling(lookback).apply(
        lambda x: np.prod(1 + x) - 1, raw=True
    )
    signal = np.sign(cum_ret)
    strat_raw = (signal * returns).dropna(how='all')
    mult_i = 0.4 / (strat_raw.std() * np.sqrt(12))
    strat_levered = strat_raw * mult_i
    port_raw = strat_levered.mean(axis=1).dropna()
    port_vol = port_raw.std() * np.sqrt(12)
    port_mult = 0.4 / port_vol
    port_levered = port_raw * port_mult
    return port_levered

# -- Loop over lookbacks and run regressions --
lookbacks = [3, 12]
for lb in lookbacks:
    port = compute_port_levered(fut_returns, lb)
    port.name = f"Port_{lb}m"

    # align portfolio and factors
    df_reg = pd.concat([port, dep_factors], axis=1).dropna()
    y = df_reg[port.name]
    X = sm.add_constant(df_reg[['Mkt_RF', 'CI_Ret']])

    # fit OLS
    model = sm.OLS(y, X).fit()

    # output summary
    print(f"\n=== {lb}-Month Lookback Regression ===")
    print(model.summary())


=== 3-Month Lookback Regression ===
                            OLS Regression Results                            
Dep. Variable:                Port_3m   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     6.115
Date:                Thu, 01 May 2025   Prob (F-statistic):            0.00280
Time:                        05:37:02   Log-Likelihood:                 119.69
No. Observations:                 153   AIC:                            -233.4
Df Residuals:                     150   BIC:                            -224.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          

### Interpretation of regression results:
>
> From the regression of 3-month lookback, we can see that the portfolio generates an excess return of ~2.28% per month with p-value of 0.017, meaning it is statistically significant at 5% confidence level. It suggests short-horizon momentum strategy is able to generate positive excess return beyond the returns caused by equity and commodity market exposure. The negative (-0.5632) coefficient on Mkt-RF factor means the TSMOM strategy has negative exposure to equity market. It means investors can use TSOMOM as partial hedge against equity market underperformance. The short-horizon TSMOM strategy also has a positive relationship to the average commodity market performance, although it is somewhat less exposed than the 12-month TSMOM strategy which has an exposure of 1.44. It tells us the strategy tracks the broad commodity market trends over the last three months. The small R-square of 0.075 means only 7.5% of monthly portfolio return variation is explained by the overall equity and commodity market exposure. The strategy also captures other idiosyncractic sources of return.
>
> Using a 12-month lookback period, we seem to get a much smaller alpha (0.0013) which is statistically insignificant (p-value is 0.883). It shows that using a longer lookback period would generate less noise. Other than that, we get similar results explained above when using 3-month lookback period, which are significant negative relationship with equity market excess return and positive relationship with overall commodity market return. In this regression, we were able to use these two factors to explain slightly more strategy return variation (27.9%).




### 5. Comparison with Demystifying Managed Futures Paper

> The 3‑ and 12‑month volatility‑targeted momentum portfolios exhibit statistically significant negative equity betas
and positive commodity betas, which is consistent with the paper's finding that CTAs load
heavily on time‑series momentum across asset classes. However, our commodity‑only portfolios have lower Sharpe ratios of about 0.98 and 0.65 at 40% vol, which is noticeably lower than the 1.5 sharpe ratio generated by the TSMOM strategy diversified across time horizon and asset classes presented in the paper, likely due to the narrower universe in our strategy. The paper also concluded using longer lookback periods would lead to higher R² (roughly 28% for 12-monthvs. ≈7.5% for 3‑month), meaning more return variation is explained by factors. Using short-term momentum would leave us with significant alpha that cannot be explained by equity and commodity market exposures.


### 6. Out-of-sample Data Analysis

In [ ]:
#6a. Long-only Stats
long_only_stats_oos = summary_stats(data_oos)

print("Stat Summary for Out-of-Sample Period (Mean Returns and Volatility shown as %)\n")
print(long_only_stats_oos)

Stat Summary for Out-of-Sample Period (Mean Returns and Volatility shown as %)

               Mean Ann Return  Volatility  Sharpe
Corn                     -4.81       24.75   -0.19
Kansas Wheat             -1.87       27.22   -0.07
Soybeans                 -1.39       23.88   -0.06
Wheat                     0.06       28.03    0.00
Brent Crude              -5.90       27.60   -0.21
WTI Crude                -6.71       29.31   -0.23
Heating Oil              -4.47       25.79   -0.17
Gasoil                   -4.63       26.96   -0.17
Natural Gas               7.62       39.31    0.19
Gasoline                 -5.35       31.43   -0.17
Cocoa                     4.97       24.61    0.20
Cotton                   -0.86       22.46   -0.04
Coffee                   -7.90       27.84   -0.28
Sugar                    -5.92       26.72   -0.22
Feeder Cattle             1.43       18.13    0.08
Live Cattle               1.68       16.73    0.10
Lean Hogs                 2.70       37.06    0.07
Go

In [ ]:
#6b: Time-series Momentum Strategy - Individual Contracts
returns1 = data_oos

results_individual_1 = []
lookbacks = [3, 12]

for lb in lookbacks:
    # compute lagged cumulative return over lookback (exclude current month)
    cum_ret = returns1.shift(1).rolling(window=lb).apply(
        lambda x: np.prod(1 + x) - 1, raw=True
    )
    # momentum signal
    signals = np.sign(cum_ret)
    # raw strategy returns per contract
    strat_raw = signals * returns1
    # leverage each contract to 40% annualized volatility
    vol_i = strat_raw.std() * np.sqrt(12)
    mult_i = 0.4 / vol_i
    strat_levered = strat_raw * mult_i

    for contract in strat_levered.columns:
        s = strat_levered[contract].dropna()
        ann_ret = s.mean() * 12 * 100
        ann_vol = s.std() * np.sqrt(12) * 100
        sharpe = ann_ret / ann_vol
        skew = s.skew()
        kurtosis = s.kurt() + 3  # convert to raw kurtosis
        results_individual_1.append({
            'Lookback': f'{lb}-month',
            'Contract': contract,
            'Annualized Return': ann_ret,
            'Annualized Volatility': ann_vol,
            'Sharpe Ratio': sharpe,
            'Skewness': skew,
            'Kurtosis': kurtosis
        })

df_output_individual_1 = pd.DataFrame(results_individual_1).set_index(['Lookback', 'Contract']).sort_index()

print('Annualized Returns and Volatility shown as %')
df_output_individual_1

Annualized Returns and Volatility shown as %


Annualized Return  Annualized Volatility  \
Lookback Contract                                                  
12-month Aluminum               14.302843                   40.0   
         Brent Crude            18.171531                   40.0   
         Cocoa                  -8.655412                   40.0   
         Coffee                -12.567340                   40.0   
         Copper                 16.302162                   40.0   
         Corn                  -19.464949                   40.0   
         Cotton                -17.440038                   40.0   
         Feeder Cattle          14.344587                   40.0   
         Gasoil                 19.275241                   40.0   
         Gasoline               14.962409                   40.0   
         Gold                   -3.133806                   40.0   
         Heating Oil            13.742995                   40.0   
         Kansas Wheat          -20.772740                   40.0   
         Lead                   -7.315089                   40.0   
         Lean Hogs             -15.889186                   40.0   
         Live Cattle            -4.631628                   40.0   
         Natural Gas            -6.742949                   40.0   
         Nickel                -25.269808                   40.0   
         SPX                    26.076615                   40.0   
         Silver                  0.840930                   40.0   
         Soybeans              -14.824760                   40.0   
         Sugar                   4.528150                   40.0   
         WTI Crude              14.593957                   40.0   
         Wheat                 -16.666408                   40.0   
         Zinc                    1.816089                   40.0   
3-month  Aluminum                1.787826                   40.0   
         Brent Crude             3.216599                   40.0   
         Cocoa                 -15.860447                   40.0   
         Coffee                  9.095834                   40.0   
         Copper                -13.630997                   40.0   
         Corn                  -11.497425                   40.0   
         Cotton                 -1.940129                   40.0   
         Feeder Cattle          16.773691                   40.0   
         Gasoil                  8.895699                   40.0   
         Gasoline                1.097853                   40.0   
         Gold                  -22.981912                   40.0   
         Heating Oil            -8.845330                   40.0   
         Kansas Wheat          -22.069869                   40.0   
         Lead                  -19.382701                   40.0   
         Lean Hogs             -11.634548                   40.0   
         Live Cattle             1.897137                   40.0   
         Natural Gas            -9.248943                   40.0   
         Nickel                  7.857229                   40.0   
         SPX                     0.324716                   40.0   
         Silver                -23.002233                   40.0   
         Soybeans               -7.230777                   40.0   
         Sugar                  -9.345136                   40.0   
         WTI Crude               7.171454                   40.0   
         Wheat                 -20.168622                   40.0   
         Zinc                    8.250105                   40.0   

                        Sharpe Ratio  Skewness   Kurtosis  
Lookback Contract                                          
12-month Aluminum           0.357571  0.149715   3.048589  
         Brent Crude        0.454288 -0.352612   3.643709  
         Cocoa             -0.216385 -0.053083   2.284983  
         Coffee            -0.314184 -2.212006  12.807768  
         Copper             0.407554 -0.513585   5.553354  
         Corn              -0.486624 -0.406834   4.148091  
       

In [ ]:
#6c: Time-series Momentum Strategy - Equal Weighted Portfolio

results_eq_wgt_1 = []
returns_1 = data_oos
for lb in lookbacks:
    # compute lagged cum ret & signal
    cum_ret = returns_1.shift(1).rolling(window=lb).apply(
        lambda x: np.prod(1 + x) - 1, raw=True
    )
    signals = np.sign(cum_ret)
    strat_raw = signals * returns_1

    # equally-weighted portfolio raw returns
    port_raw = strat_raw.mean(axis=1)

    # leverage portfolio to 40% annualized volatility
    port_vol = port_raw.std() * np.sqrt(12)
    port_mult = 0.4 / port_vol
    port_levered = port_raw * port_mult

    # compute metrics
    ann_ret = port_levered.mean() * 12
    ann_vol = port_levered.std() * np.sqrt(12)
    sharpe = ann_ret / ann_vol
    skew = port_levered.skew()
    kurtosis = port_levered.kurt() + 3
    results_eq_wgt_1.append({
        'Lookback': f'{lb}-month',
        'Annualized Return': ann_ret * 100,
        'Annualized Volatility': ann_vol * 100,
        'Sharpe Ratio': sharpe,
        'Skewness': skew,
        'Kurtosis': kurtosis
    })

print("Annualized returns and volatility are shown in %")
df_output_eq_wgt_1 = pd.DataFrame(results_eq_wgt_1).set_index('Lookback')
df_output_eq_wgt_1

Annualized returns and volatility are shown in %


,Annualized Return,Annualized Volatility,Sharpe Ratio,Skewness,Kurtosis
Lookback,,,,,
3-month,-19.477069,40.0,-0.486927,-0.189812,3.223537
12-month,-5.039929,40.0,-0.125998,-0.117010,3.802338


In [ ]:
#6d: Two-factor regression on portfolio returns

file_path_1 = "/content/sample_data/Research Project Data.xlsx"
data2 = pd.read_excel(
    file_path,
    sheet_name='Futures Out of Sample',
    usecols='A:AB'
)


data2['Date'] = pd.to_datetime(data2['YearMonth'].astype(str), format='%Y%m')
data2.set_index('Date', inplace=True)
data2= data2.drop(columns=['YearMonth'])

data2 = data2.rename(columns={data2.columns[-2]: 'Mkt_RF', data2.columns[-1]: 'CI_Ret'})

# Split futures returns and factors
fut_returns_1 = data2.drop(columns=['Mkt_RF', 'CI_Ret','SPX'])
dep_factors_1 = data2[['Mkt_RF', 'CI_Ret']]

def compute_port_levered(returns, lookback):
    cum_ret = returns.shift(1).rolling(lookback).apply(
        lambda x: np.prod(1 + x) - 1, raw=True
    )
    signal = np.sign(cum_ret)
    strat_raw = (signal * returns).dropna(how='all')
    mult_i = 0.4 / (strat_raw.std() * np.sqrt(12))
    strat_levered = strat_raw * mult_i
    port_raw = strat_levered.mean(axis=1).dropna()
    port_vol = port_raw.std() * np.sqrt(12)
    port_mult = 0.4 / port_vol
    port_levered = port_raw * port_mult
    return port_levered

# -- Loop over lookbacks and run regressions --
lookbacks = [3, 12]
for lb in lookbacks:
    port = compute_port_levered(fut_returns_1, lb)
    port.name = f"Port_{lb}m"

    # align portfolio and factors
    df_reg = pd.concat([port, dep_factors_1], axis=1).dropna()
    y = df_reg[port.name]
    X = sm.add_constant(df_reg[['Mkt_RF', 'CI_Ret']])

    # fit OLS
    model = sm.OLS(y, X).fit()

    # output summary
    print(f"\n=== {lb}-Month Lookback Regression ===")
    print(model.summary())


=== 3-Month Lookback Regression ===
                            OLS Regression Results                            
Dep. Variable:                Port_3m   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.567
Date:                Thu, 01 May 2025   Prob (F-statistic):              0.215
Time:                        05:38:58   Log-Likelihood:                 62.023
No. Observations:                  81   AIC:                            -118.0
Df Residuals:                      78   BIC:                            -110.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -

### 7. Comparison with In-Sample Results

> For long-only stat, most contracts did remarkably better in in-sample period than out-of-sample. Mean return and Sharpe ratio were higher for OOS, although volatility in OOS period was somewhat lower than in-sample.
> For leveraged portfolio, most individual contracts were generating negative returns and Sharpe ratio, representing poor performance and risk-return tradeoff in OOS period. Similar results can be observed in equal weighted portfolio too. The strategy generated 22.43% of annalized return in sample-period compared to -5.04% in OOS period. Using a 12-month lookback period led to less disperse outcomes, which can be observed in both in-sample and OOS period. Volatility targeting works in-sample (Sharpe ratio of 0.93/0.63) but fails OOS (Sharpe ratio of –0.41/–0.03).
> In terms of the regression result, using a 12-month lookback in the 2-factor model, resulted in a slightly lower R-sqaure (0.279 vs 0.29) in in-sample period, which means less return was explained equity and commodity market overall. What's noteworthy is that the coefficients for market excess return and commodity are opposite in signs. For in-sample period, the regression results told us strategy returns had negative beta on equity market excess return and positive beta on commodity market. In OOS period, we saw an insignificant (p-value = 0.659) positive coefficient for Mkt-Rf and a signficant (p-value = 0.000) negative coefficient for CI_Ret.
> Due to the shift in market regime and whipsaw market, we see the trend-following feature of TSMOM lag in performance. Correlation with equity and commodity market can also shift over time.

## PART 2 – AQR TSMOM Integration

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
import os

# Load the data
file_path = '/content/sample_data/Research Project Data.xlsx'

raw_data = pd.read_excel(file_path, sheet_name=None)
aqr_tsmom = raw_data['TSMOM Data']
aqr_tsmom_oos = raw_data['TSMOM Out of Sample']

# Format date columns
aqr_tsmom['Date'] = pd.to_datetime(aqr_tsmom['Date'])
aqr_tsmom.set_index('Date', inplace=True)
aqr_tsmom_oos['Date'] = pd.to_datetime(aqr_tsmom_oos['Date'])
aqr_tsmom_oos.set_index('Date', inplace=True)

# Rename relevant columns to standard format
aqr_combined_in = aqr_tsmom[['TSMOM', 'Rm-Rf']].copy()
aqr_combined_in.columns = ['TSMOM', 'Mkt_RF']
aqr_combined_oos = aqr_tsmom_oos[['TSMOM', 'Rm-Rf']].copy()
aqr_combined_oos.columns = ['TSMOM', 'Mkt_RF']

# Portfolio combination function
def combine_portfolios(df, w):
    return w * df['TSMOM'] + (1 - w) * df['Mkt_RF']

# Portfolio statistics function
def port_stats(ts):
    return {
        'Mean Ann Return': ts.mean() * 12,
        'Volatility': ts.std() * np.sqrt(12),
        'Sharpe': (ts.mean() * 12) / (ts.std() * np.sqrt(12)),
        'Skew': skew(ts),
        'Kurtosis': kurtosis(ts)
    }

# 1. Create 50/50 portfolios
combo_5050_in = combine_portfolios(aqr_combined_in, 0.5)
combo_5050_oos = combine_portfolios(aqr_combined_oos, 0.5)

# 2. Stats
aqr_results_in = {
    'Mkt-RF': port_stats(aqr_combined_in['Mkt_RF']),
    'TSMOM': port_stats(aqr_combined_in['TSMOM']),
    'Combo 50/50': port_stats(combo_5050_in)
}

aqr_results_oos = {
    'Mkt-RF': port_stats(aqr_combined_oos['Mkt_RF']),
    'TSMOM': port_stats(aqr_combined_oos['TSMOM']),
    'Combo 50/50': port_stats(combo_5050_oos)
}

# 3. Optimal TSMOM allocation for max Sharpe (out-of-sample)
best_sharpe = -np.inf
best_weight = 0
for w in range(0, 101):
    p = combine_portfolios(aqr_combined_oos, w / 100)
    s = port_stats(p)['Sharpe']
    if s > best_sharpe:
        best_sharpe = s
        best_weight = w

# 4. Print Results
print("In-Sample Portfolio Stats:")
for k, v in aqr_results_in.items():
    print(f"{k}:")
    for stat, value in v.items():
        print(f"  {stat}: {value:.4f}")

print("\nOut-of-Sample Portfolio Stats:")
for k, v in aqr_results_oos.items():
    print(f"{k}:")
    for stat, value in v.items():
        print(f"  {stat}: {value:.4f}")

print(f"\nOptimal TSMOM allocation for max Sharpe (Out of Sample): {best_weight}%")

print("\nBased on the in-sample results, the TSMOM strategy delivered strong standalone performance with an annualized Sharpe ratio of 0.92, significantly higher than the equity market Sharpe ratio of 0.44.")
print("Furthermore, a 50/50 portfolio combining equities and TSMOM improved the Sharpe to 0.96, showing clear diversification benefits and better risk-adjusted returns.")
print("However, in the out-of-sample period, TSMOM’s performance dropped considerably, with a near-zero Sharpe ratio of 0.05. In contrast, the equity market Sharpe ratio increased to 1.16, outperforming TSMOM by a large margin. The 50/50 blend still maintained a decent Sharpe of 0.96, but this was largely driven by equities.")
print("The optimal allocation to TSMOM that maximized out-of-sample Sharpe was 24%, suggesting that a smaller exposure could still provide some diversification benefit—especially considering the positive skew and lower kurtosis of the blended portfolio compared to equities alone.")
print("\nConclusion: While in-sample results favor a sizable TSMOM allocation, the strategy's underperformance out-of-sample urges caution. I would include TSMOM in the portfolio, but at a reduced allocation (~20–25%) to hedge against equity risk and benefit from potential diversification, while managing the risk of diminished returns in changing market regimes.")


In-Sample Portfolio Stats:
Mkt-RF:
  Mean Ann Return: 0.0711
  Volatility: 0.1603
  Sharpe: 0.4436
  Skew: -0.8757
  Kurtosis: 2.5115
TSMOM:
  Mean Ann Return: 0.1317
  Volatility: 0.1438
  Sharpe: 0.9155
  Skew: -0.3409
  Kurtosis: 1.9344
Combo 50/50:
  Mean Ann Return: 0.1014
  Volatility: 0.1061
  Sharpe: 0.9559
  Skew: -0.4807
  Kurtosis: 2.0780

Out-of-Sample Portfolio Stats:
Mkt-RF:
  Mean Ann Return: 0.1580
  Volatility: 0.1366
  Sharpe: 1.1567
  Skew: -0.4561
  Kurtosis: 2.0633
TSMOM:
  Mean Ann Return: 0.0078
  Volatility: 0.1573
  Sharpe: 0.0495
  Skew: 0.7473
  Kurtosis: 2.0257
Combo 50/50:
  Mean Ann Return: 0.0829
  Volatility: 0.0861
  Sharpe: 0.9630
  Skew: 0.1204
  Kurtosis: 0.3472

Optimal TSMOM allocation for max Sharpe (Out of Sample): 24%

Based on the in-sample results, the TSMOM strategy delivered strong standalone performance with an annualized Sharpe ratio of 0.92, significantly higher than the equity market Sharpe ratio of 0.44.
Furthermore, a 50/50 portfolio c